In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess
%matplotlib inline

In [14]:
subprocess.run('scavetool x -o tictoc/results/tictoc.csv tictoc/results/*.sca tictoc/results/*.vec', shell=True)

CompletedProcess(args='scavetool x -o tictoc/results/tictoc.csv tictoc/results/*.sca tictoc/results/*.vec', returncode=0)

In [25]:
# tictoc.csv was built with the following command executed from the tictoc root:
#
# > scavetool x -o results/tictoc.csv results/*.vec results/*.sca
#

tictoc = pd.read_csv('tictoc/results/tictoc.csv')

In [26]:
tictoc.head()

,run,type,module,name,attrname,attrvalue,value,vectime,vecvalue
0,WheelNetwork-5-0-20180710-13:23:40-23746,runattr,NaN,NaN,configname,WheelNetwork-5,NaN,NaN,NaN
1,WheelNetwork-5-0-20180710-13:23:40-23746,runattr,NaN,NaN,datetime,20180710-13:23:40,NaN,NaN,NaN
2,WheelNetwork-5-0-20180710-13:23:40-23746,runattr,NaN,NaN,experiment,WheelNetwork-5,NaN,NaN,NaN
3,WheelNetwork-5-0-20180710-13:23:40-23746,runattr,NaN,NaN,inifile,omnetpp.ini,NaN,NaN,NaN
4,WheelNetwork-5-0-20180710-13:23:40-23746,runattr,NaN,NaN,iterationvars,NaN,NaN,NaN,NaN


In [27]:
vectors = tictoc[tictoc['type']=='vector']
vectors

,run,type,module,name,attrname,attrvalue,value,vectime,vecvalue
15,WheelNetwork-5-0-20180710-13:23:40-23746,vector,WheelNetwork.tic[5],hopCount:vector,NaN,NaN,NaN,0.005 0.045 0.09,1 1 8
19,WheelNetwork-5-0-20180710-13:23:40-23746,vector,WheelNetwork.tic[4],hopCount:vector,NaN,NaN,NaN,0.04 0.135 0.175 0.23 0.29,2 7 4 9 3


In [23]:
# list modules
set(tictoc.module.unique())

{'WheelNetwork.tic[4]', 'WheelNetwork.tic[5]', nan}

In [63]:
# Explore all rows with hopCount stats and show only meaningful fields:
hop_count_stats = tictoc[(tictoc['type']=='statistic') & (tictoc['name']=='hopCount:stats')][
    ['module', 'count', 'sumweights', 'mean', 'stddev', 'min', 'max']
]
hop_count_stats

,module,count,sumweights,mean,stddev,min,max
113,Tictoc.tic[0],1242.0,NaN,7.318841,6.136622,1.0,58.0
117,Tictoc.tic[1],1141.0,NaN,2.652936,2.566410,1.0,22.0
121,Tictoc.tic[2],1176.0,NaN,7.827381,6.600578,1.0,43.0
125,Tictoc.tic[3],1211.0,NaN,7.294798,6.229877,1.0,49.0
129,Tictoc.tic[4],1223.0,NaN,2.704824,2.682678,1.0,26.0
133,Tictoc.tic[5],1199.0,NaN,7.179316,6.385792,1.0,59.0


In [24]:
# Describe mean values of the hopCount statistics:
hop_count_stats['mean'].describe()

NameError: name 'hop_count_stats' is not defined

In [65]:
#
# Since some columns may be treated as strings while being numbers or arrays, we need
# a more complex way to read the CSV. To do it, we define two additional methods:
#
def parse_if_number(s):
    try:
        return float(s)
    except:
        return True if s=='true' else False if s == 'false' else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

In [68]:
#
# And now we read CSV in the right way. We could also use 'usecols=["name", ...]'
# to load only the columns we actualy need, but not this time.
#
tictoc = pd.read_csv('results/tictoc.csv', converters={
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})
tictoc[
    (tictoc.name.str.endswith('histogram') | (tictoc['type'] == 'vector')) 
    & (tictoc['type'] != 'attr')
][[
    'type', 'module', 'name', 'type', 'count', 'mean', 'stddev', 'min', 'max', 
    'binedges', 'binvalues', 'vectime', 'vecvalue',
]]

,type,module,name,type,count,mean,stddev,min,max,binedges,binvalues,vectime,vecvalue
137,histogram,Tictoc.tic[1],hopCount:histogram,histogram,1141.0,2.652936,2.56641,1.0,22.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 551.0, 206.0, 88.0, 121.0, 32.0, 58.0, 2...",None,None
141,vector,Tictoc.tic[1],hopCount:vector,vector,NaN,NaN,NaN,NaN,NaN,None,None,"[0.1, 2.5, 6.3, 8.3, 11.9, 12.7, 16.7, 17.5, 1...","[1.0, 1.0, 16.0, 4.0, 1.0, 1.0, 1.0, 2.0, 3.0,..."
145,vector,Tictoc.tic[4],hopCount:vector,vector,NaN,NaN,NaN,NaN,NaN,None,None,"[2.0, 3.2, 4.0, 6.8, 8.4, 11.8, 18.0, 19.6, 21...","[10.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 13.0, 2.0..."
149,vector,Tictoc.tic[3],hopCount:vector,vector,NaN,NaN,NaN,NaN,NaN,None,None,"[3.1, 7.9, 9.1, 11.7, 12.3, 16.3, 17.3, 19.7, ...","[6.0, 2.0, 7.0, 3.0, 3.0, 11.0, 6.0, 1.0, 2.0,..."
153,vector,Tictoc.tic[5],hopCount:vector,vector,NaN,NaN,NaN,NaN,NaN,None,None,"[3.9, 4.7, 7.7, 26.3, 27.1, 27.9, 32.1, 38.1, ...","[7.0, 3.0, 9.0, 2.0, 5.0, 3.0, 7.0, 6.0, 5.0, ..."


In [70]:
tictoc[tictoc.type == 'scalar'][['module', 'name', 'value']]

,module,name,value
17,Tictoc.tic[0],hopCount:timeavg,7.608896
21,Tictoc.tic[0],hopCount:min,1.000000
25,Tictoc.tic[0],hopCount:max,58.000000
29,Tictoc.tic[0],hopCount:avg,7.318841
33,Tictoc.tic[1],hopCount:timeavg,2.590036
37,Tictoc.tic[1],hopCount:min,1.000000
41,Tictoc.tic[1],hopCount:max,22.000000
45,Tictoc.tic[1],hopCount:avg,2.652936
49,Tictoc.tic[2],hopCount:timeavg,7.856525
53,Tictoc.tic[2],hopCount:min,1.000000
